# Huấn luyện mô hình dự đoán khoảng giá bằng LightGBM Quantile và Conformal Prediction

## Chuẩn bị thư viện và load dữ liệu đã xử lý

In [7]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
from datetime import date
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# =============================
# Load processed data
# =============================
print("\n\u2714\ufe0f Loading processed data...")
X_train_scaled = np.load('processed/X_train_scaled.npy')
X_val_scaled = np.load('processed/X_val_scaled.npy')
X_test_scaled = np.load('processed/X_test_scaled.npy')
y_train = np.load('processed/y_train.npy')
y_val = np.load('processed/y_val.npy')

feature_columns = joblib.load('processed/feature_columns.pkl')



✔️ Loading processed data...


## Tạo đặc trưng nâng cao (Feature Engineering)

In [8]:
# Hàm tạo đặc trưng nâng cao từ dữ liệu đã chuẩn hóa
def create_advanced_features(X_scaled, feature_names):
    X_df = pd.DataFrame(X_scaled, columns=feature_names)
    current_year = date.today().year

    # Diện tích liên quan
    if 'sqft' in X_df and 'sqft_lot' in X_df:
        X_df['sqft_ratio'] = X_df['sqft'] / (X_df['sqft_lot'] + 1)
        X_df['sqft_lot_log'] = np.log1p(np.abs(X_df['sqft_lot']))
        X_df['sqft_log'] = np.log1p(np.abs(X_df['sqft']))

    # Giá trị đất và xây dựng
    if 'land_val' in X_df and 'imp_val' in X_df:
        X_df['total_val'] = X_df['land_val'] + X_df['imp_val']
        X_df['imp_land_ratio'] = X_df['imp_val'] / (X_df['land_val'] + 1)
        X_df['total_val_log'] = np.log1p(np.abs(X_df['total_val']))
        X_df['land_val_per_sqft'] = X_df['land_val'] / (X_df['sqft_lot'] + 1)

    # Tương tác giữa phòng ngủ và diện tích
    if 'beds' in X_df and 'sqft' in X_df:
        X_df['sqft_per_bed'] = X_df['sqft'] / (X_df['beds'] + 1)
        X_df['beds_sqft_interaction'] = X_df['beds'] * X_df['sqft']

    # Tổng số phòng tắm quy đổi
    if 'bath_full' in X_df and 'bath_3qtr' in X_df and 'bath_half' in X_df:
        X_df['total_bath'] = X_df['bath_full'] + 0.75 * X_df['bath_3qtr'] + 0.5 * X_df['bath_half']
        X_df['bath_per_bed'] = X_df['total_bath'] / (X_df['beds'] + 1)

    # Tuổi nhà và năm cải tạo
    if 'year_built' in X_df:
        X_df['house_age'] = current_year - X_df['year_built']
        X_df['house_age_squared'] = X_df['house_age'] ** 2
        if 'year_reno' in X_df:
            X_df['years_since_reno'] = np.where(X_df['year_reno'] > 0, current_year - X_df['year_reno'], X_df['house_age'])

    # Tổng điểm view
    view_cols = [col for col in X_df if col.startswith('view_')]
    if view_cols:
        X_df['total_view_score'] = X_df[view_cols].sum(axis=1)
        X_df['has_view'] = (X_df['total_view_score'] > 0).astype(int)

    # Diện tích garage
    if 'gara_sqft' in X_df and 'sqft' in X_df:
        X_df['garage_ratio'] = X_df['gara_sqft'] / (X_df['sqft'] + 1)
        X_df['has_garage'] = (X_df['gara_sqft'] > 0).astype(int)

    # Tọa độ địa lý
    if 'latitude' in X_df and 'longitude' in X_df:
        X_df['lat_long_interaction'] = X_df['latitude'] * X_df['longitude']
        X_df['distance_from_center'] = np.sqrt((X_df['latitude'] - X_df['latitude'].mean())**2 + (X_df['longitude'] - X_df['longitude'].mean())**2)

    # Đánh giá tổng thể
    if 'grade' in X_df and 'condition' in X_df:
        X_df['grade_condition_interaction'] = X_df['grade'] * X_df['condition']
        X_df['quality_score'] = X_df['grade'] + X_df['condition']

    return X_df.values


In [9]:
# Áp dụng tạo đặc trưng cho các tập train, val, test
X_train = create_advanced_features(X_train_scaled, feature_columns)
X_val = create_advanced_features(X_val_scaled, feature_columns)
X_test = create_advanced_features(X_test_scaled, feature_columns)

## Huấn luyện mô hình LightGBM Quantile

In [10]:
# Huấn luyện 2 mô hình LGBM để học phân vị 5% và 95%
print("\n\u2714\ufe0f Training quantile regressors...")

lgb_params = {
    'objective': 'quantile',
    'n_estimators': 1200,
    'learning_rate': 0.02,
    'num_leaves': 64,
    'max_depth': 7,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1.0,
    'reg_lambda': 1.0,
    'random_state': 42
}

# Mô hình cho phân vị thấp (5%)
lgb_5 = lgb.LGBMRegressor(**lgb_params, alpha=0.05)

# Mô hình cho phân vị cao (95%)
lgb_95 = lgb.LGBMRegressor(**lgb_params, alpha=0.95)

# Huấn luyện
lgb_5.fit(X_train, y_train)
lgb_95.fit(X_train, y_train)

# Dự đoán khoảng trên tập validation
val_l = lgb_5.predict(X_val)
val_u = lgb_95.predict(X_val)



✔️ Training quantile regressors...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043363 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8061
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 71
[LightGBM] [Info] Start training from score 185000.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

## Hiệu chỉnh bằng Conformal Prediction

In [11]:
# Tính residual và hiệu chỉnh bằng Conformal Prediction
residuals = np.maximum(0, val_l - y_val) + np.maximum(0, y_val - val_u)
q_hat = np.quantile(residuals, 0.9)

# Khoảng dự đoán hiệu chỉnh
val_l_adj = val_l - q_hat
val_u_adj = val_u + q_hat

# Đánh giá độ phủ (coverage)
coverage = np.mean((y_val >= val_l_adj) & (y_val <= val_u_adj))


## Tính các độ đo đánh giá

In [12]:
# Hàm tính Winkler Score (đánh giá khoảng dự đoán tốt cả chiều rộng lẫn độ chính xác)
def winkler_score(y_true, l, u, alpha=0.1):
    width = u - l
    penalty = np.where(y_true < l, (2/alpha)*(l - y_true),
              np.where(y_true > u, (2/alpha)*(y_true - u), 0))
    return width + penalty

# Tính các chỉ số
val_winkler = np.mean(winkler_score(y_val, val_l_adj, val_u_adj, alpha=0.1))
mean_width = np.mean(val_u_adj - val_l_adj)
val_mae = mean_absolute_error(y_val, (val_l_adj + val_u_adj) / 2)
val_rmse = np.sqrt(mean_squared_error(y_val, (val_l_adj + val_u_adj) / 2))

# In kết quả
print(f"\nCoverage: {coverage:.3f}")
print(f"Winkler Score: {val_winkler:,.2f}")
print(f"MAE: {val_mae:,.2f}")
print(f"RMSE: {val_rmse:,.2f}")
print(f"Mean Interval Width: {mean_width:,.2f}")
print(f"q_hat: {q_hat:.2f}")



Coverage: 0.900
Winkler Score: 341,850.64
MAE: 63,012.56
RMSE: 116,814.94
Mean Interval Width: 247,135.10
q_hat: 5863.87


## Dự đoán trên tập test và tạo submission

In [13]:
# Dự đoán trên tập test và lưu kết quả submission
print("\n\u2714\ufe0f Generating final predictions...")

# Áp dụng khoảng hiệu chỉnh
test_l = lgb_5.predict(X_test) - q_hat
test_u = lgb_95.predict(X_test) + q_hat

# Lấy ID gốc từ file test
test_df = pd.read_csv("dataset/test.csv")

# Tạo submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'pi_lower': test_l,
    'pi_upper': test_u
})
submission.to_csv("submission.csv", index=False)

print("\n\u2714\ufe0f Submission saved as 'submission.csv'")


✔️ Generating final predictions...

✔️ Submission saved as 'submission.csv'
